Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [11]:
valid_size = 2000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99998000  kropotkin zeno repudiated the omnipotence of the state its inte
2000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [12]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [15]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

[' kropotkin ', 'agencies in', 'may be conv', 'ar of many ', 'ress of bou', 'ew of ausch', 'est of it i', 'h for what ', 'ital city t', 'arinagu are', 'any bass sp', 'qualified t', 'o david ord', ' charismati', 'heir morale', 'p their car', 'reated as u', 'o responsib', 'e c s one e', 'than nothin', 'united nati', 's of cryopr', 'ar computer', 'en six eigh', 'sis within ', 'users or re', 'ues to beli', 'k county on', 'in the west', 'mers can se', ' in a gener', 'urplus to r', 'opular in j', 'ays and int', 'evolutionar', 'il standard', ' coupling t', 'thrive unde', 'hysics math', 'the garden ', 'ering const', 'an subconti', 'ith amplitu', 'representin', 'fferent lan', ' have had b', ' press one ', 'veloping a ', 'zero cm thi', ' see it eit', 'economic po', 'ing on whic', 'pany was or', 'ko higashik', 'ight genera', 'in work sig', ' a watch to', 'el ed encyc', ' aviation a', ' certifying', ' on this da', 'er state mo', 'f devotiona', 'es in vlsi ']
[' zeno repud', 'n operation', 'veyed b

In [16]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [17]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [18]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298339 learning rate: 10.000000
Minibatch perplexity: 27.07
wjnx euezgr eicavwlhhcdpmicnromphplle ifquwcrliehejofetbdseaey avaffahvtewbxnzna
mtlttw nboi itxyrhooklkx  hhvghr  n  mdi  c  knffusr s es  smimfrjwpxmss a yktgm
is i vrcqryliikkhu pitmrwsdhzeitt uve eynqpwinofnhgohanycctrgxddshr lpvarvt dci 
kx xx xtaz anfq jwpgt  wirope tcrnhaispgoivtgoyjs jpoawxxuuurexzcif aqtw lkapxoi
lb  qqfviodrhc nfyuesr e syd  l ec   alhnotkebdyolzl ugvybntbl e mj ylruovdmygfx
Validation set perplexity: 20.37
Average loss at step 100: 2.578494 learning rate: 10.000000
Minibatch perplexity: 10.75
Validation set perplexity: 11.47
Average loss at step 200: 2.245175 learning rate: 10.000000
Minibatch perplexity: 8.48
Validation set perplexity: 9.24
Average loss at step 300: 2.095390 learning rate: 10.000000
Minibatch perplexity: 8.23
Validation set perplexity: 8.61
Average loss at step 400: 1.998455 learning rate: 10.000000
Minibatch perplexity: 6.90
Validation set per

Validation set perplexity: 4.93
Average loss at step 4500: 1.603086 learning rate: 10.000000
Minibatch perplexity: 5.00
Validation set perplexity: 4.90
Average loss at step 4600: 1.635459 learning rate: 10.000000
Minibatch perplexity: 4.51
Validation set perplexity: 4.94
Average loss at step 4700: 1.620300 learning rate: 10.000000
Minibatch perplexity: 4.75
Validation set perplexity: 4.86
Average loss at step 4800: 1.632817 learning rate: 10.000000
Minibatch perplexity: 4.97
Validation set perplexity: 4.96
Average loss at step 4900: 1.625715 learning rate: 10.000000
Minibatch perplexity: 4.42
Validation set perplexity: 4.94
Average loss at step 5000: 1.601933 learning rate: 1.000000
Minibatch perplexity: 5.45
viantine tran only as of the used cohly mongus and moscay beap became some which
ball of per frim years obn its kamirjo and him on the flored josas was merilogy 
lic the disproppency it the east to destrimoristone of the miss killion products
chils economic come cheisuriods conser

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [17]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Concatenate parameters  
  sx = tf.concat([ix, fx, cx, ox], 1)
  sm = tf.concat([im, fm, cm, om], 1)
  sb = tf.concat([ib, fb, cb, ob], 1)
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    smatmul = tf.matmul(i, sx) + tf.matmul(o, sm) + sb
    smatmul_input, smatmul_forget, update, smatmul_output = tf.split(smatmul, 4, 1)
    input_gate = tf.sigmoid(smatmul_input)
    forget_gate = tf.sigmoid(smatmul_forget)
    output_gate = tf.sigmoid(smatmul_output)
    #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels, 0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [18]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.297277 learning rate: 10.000000
Minibatch perplexity: 27.04
w nkzty ezpsesmju  t lofziu  ahk j is  riptuedr z enowm  lhowdjq qedhe  q nuccwm
t eosj omriv nwrvkzod xbre a   hqryebioja xtseeqrjjpmnfxirytanez rtmac rqe  ot o
eyhmm ke kedacefdciaclmh  srvlkzgmg actkzjqs g sodz lr   egsiseaikiekydnl ix wpq
knaxkpltet np skbaigef sbiwg m eo jh ag dqlnvdd  ja czge fhg gqsxy hq  oltwnroi 
jordeb hqjfgk z nmnkyvol tzznhzv yjiaxx ttkawbjoy e na ewb ehn l j  fjezttf bcue
Validation set perplexity: 20.12
Average loss at step 100: 2.582882 learning rate: 10.000000
Minibatch perplexity: 11.17
Validation set perplexity: 11.31
Average loss at step 200: 2.245153 learning rate: 10.000000
Minibatch perplexity: 8.26
Validation set perplexity: 8.78
Average loss at step 300: 2.085276 learning rate: 10.000000
Minibatch perplexity: 6.44
Validation set perplexity: 8.10
Average loss at step 400: 2.036

Validation set perplexity: 4.84
Average loss at step 4300: 1.618395 learning rate: 10.000000
Minibatch perplexity: 5.67
Validation set perplexity: 4.90
Average loss at step 4400: 1.607563 learning rate: 10.000000
Minibatch perplexity: 5.02
Validation set perplexity: 4.86
Average loss at step 4500: 1.642783 learning rate: 10.000000
Minibatch perplexity: 5.27
Validation set perplexity: 4.88
Average loss at step 4600: 1.622379 learning rate: 10.000000
Minibatch perplexity: 5.52
Validation set perplexity: 4.79
Average loss at step 4700: 1.622949 learning rate: 10.000000
Minibatch perplexity: 4.97
Validation set perplexity: 4.82
Average loss at step 4800: 1.609234 learning rate: 10.000000
Minibatch perplexity: 4.87
Validation set perplexity: 4.75
Average loss at step 4900: 1.615899 learning rate: 10.000000
Minibatch perplexity: 5.15
Validation set perplexity: 4.66
Average loss at step 5000: 1.615142 learning rate: 1.000000
Minibatch perplexity: 4.87
variest of wight deglaw about br one two 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [19]:
batch_size=64
num_unrollings=10
bigram_size = vocabulary_size * vocabulary_size

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, bigram_size), dtype=np.float)
    for b in range(self._batch_size):
      first_char = self._text[self._cursor[b]]
      if self._cursor[b] + 1 == self._text_size:
        second_char = ' '
      else:
        second_char = self._text[self._cursor[b] + 1]
      batch[b, char2id(first_char) * vocabulary_size + char2id(second_char)] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bigram_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return ['({0},{1})'.format(id2char(c//vocabulary_size), id2char(c % vocabulary_size))
          for c in np.argmax(probabilities, 1)]

def bigram_first_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c//vocabulary_size)
          for c in np.argmax(probabilities, 1)]

def bigram_batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigram_characters(b))]
  return s

bigram_train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
bigram_valid_batches = BigramBatchGenerator(valid_text, 1, 1)

# output each bigram instead of a single char
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))

['( ,k)(k,r)(r,o)(o,p)(p,o)(o,t)(t,k)(k,i)(i,n)(n, )( ,z)', '(a,g)(g,e)(e,n)(n,c)(c,i)(i,e)(e,s)(s, )( ,i)(i,n)(n, )', '(m,a)(a,y)(y, )( ,b)(b,e)(e, )( ,c)(c,o)(o,n)(n,v)(v,e)', '(a,r)(r, )( ,o)(o,f)(f, )( ,m)(m,a)(a,n)(n,y)(y, )( ,o)', '(r,e)(e,s)(s,s)(s, )( ,o)(o,f)(f, )( ,b)(b,o)(o,u)(u,l)', '(e,w)(w, )( ,o)(o,f)(f, )( ,a)(a,u)(u,s)(s,c)(c,h)(h,w)', '(e,s)(s,t)(t, )( ,o)(o,f)(f, )( ,i)(i,t)(t, )( ,i)(i,t)', '(h, )( ,f)(f,o)(o,r)(r, )( ,w)(w,h)(h,a)(a,t)(t, )( ,s)', '(i,t)(t,a)(a,l)(l, )( ,c)(c,i)(i,t)(t,y)(y, )( ,t)(t,o)', '(a,r)(r,i)(i,n)(n,a)(a,g)(g,u)(u, )( ,a)(a,r)(r,e)(e, )', '(a,n)(n,y)(y, )( ,b)(b,a)(a,s)(s,s)(s, )( ,s)(s,p)(p,e)', '(q,u)(u,a)(a,l)(l,i)(i,f)(f,i)(i,e)(e,d)(d, )( ,t)(t,o)', '(o, )( ,d)(d,a)(a,v)(v,i)(i,d)(d, )( ,o)(o,r)(r,d)(d,e)', '( ,c)(c,h)(h,a)(a,r)(r,i)(i,s)(s,m)(m,a)(a,t)(t,i)(i,c)', '(h,e)(e,i)(i,r)(r, )( ,m)(m,o)(o,r)(r,a)(a,l)(l,e)(e, )', '(p, )( ,t)(t,h)(h,e)(e,i)(i,r)(r, )( ,c)(c,a)(a,r)(r,d)', '(r,e)(e,a)(a,t)(t,e)(e,d)(d, )( ,a)(a,s)(s, )( ,u)(u,k

In [22]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,bigram_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels,0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, bigram_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [23]:
def bigram_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, bigram_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def bigram_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, bigram_size])
  return b/np.sum(b, 1)[:,None]

In [24]:
num_steps = 7001
summary_frequency = 100

def exec_graph_bigram(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = bigram_train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = bigram_sample(bigram_random_distribution())
            bigram_sentence = bigram_characters(feed)[0]
            sentence = bigram_first_characters(feed)[0]
            reset_sample_state.run()
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = bigram_sample(prediction)
              bigram_sentence += bigram_characters(feed)[0]
              sentence += bigram_first_characters(feed)[0]
            print('bigrams:', bigram_sentence)
            print('chars:', sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
          b = bigram_valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))
        
%time exec_graph_bigram(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 6.591044 learning rate: 10.000000
Minibatch perplexity: 728.54
bigrams: (z,s)(h,l)(y,w)(c,b)( ,o)(b,b)(x, )(l,e)(h,r)(o,m)(c,l)(f,e)(g,c)(o,q)(b,l)( ,f)(f,d)(p,z)(u,i)(a,y)(b,d)(n,v)(r,p)(l,b)(f,u)(x,x)(i,u)(p,x)(k,g)(s,b)(j,l)(c,l)(w,u)(t,o)(e,b)(t,a)(r,m)(u,c)(s,i)(h,i)(n,i)(x,d)(l,r)(o,e)(h,v)(f,r)( ,c)(f,x)(k,s)(a,g)(a,t)(i,r)(c,s)(z,p)(j,p)( ,w)(n,u)(q, )(f,c)(d,d)(f,j)(g,a)(x,a)(e,g)(c,a)(y,r)(q,i)(n,b)(j, )(e,v)(c,f)(h,g)(q,o)(m,u)(f,f)(n,l)(a,f)(v, )(x,r)(p,e)
chars: zhyc bxlhocfgob fpuabnrlfxipksjcwtetrushnxlohf fkaaiczj nqfdfgxecyqnjechqmfnavxp
bigrams: (s,z)(c,w)(l,x)(f,u)(h,o)(b,s)(v,y)(k,m)(r,x)(k,q)(h,g)(h,b)(n,n)(i,n)(y,p)(r, )(r,l)(a,g)(f,s)(e,e)(j,j)(b,h)(b,v)(m,y)(i,l)(a,l)(w,k)(a, )( , )(j,c)(s, )(t,d)(q,x)(o,m)(b,m)(c,g)(u,o)(d,m)(t,r)(g,d)(e,k)(w,u)(b, )(c,t)(c,z)(i,g)( ,l)(s,l)(c,k)(q,e)(c,i)(h,n)(p,n)(t,x)(q,k)(g,s)(w,s)(q, )(k, )(d,t)(y,m)(v,i)(x,h)(r,i)

Validation set perplexity: 5.25
Average loss at step 2100: 1.650484 learning rate: 10.000000
Minibatch perplexity: 4.80
Validation set perplexity: 5.06
Average loss at step 2200: 1.620249 learning rate: 10.000000
Minibatch perplexity: 4.83
Validation set perplexity: 5.07
Average loss at step 2300: 1.604523 learning rate: 10.000000
Minibatch perplexity: 5.20
Validation set perplexity: 4.98
Average loss at step 2400: 1.639684 learning rate: 10.000000
Minibatch perplexity: 4.77
Validation set perplexity: 4.90
Average loss at step 2500: 1.635950 learning rate: 10.000000
Minibatch perplexity: 5.23
Validation set perplexity: 5.02
Average loss at step 2600: 1.615664 learning rate: 10.000000
Minibatch perplexity: 4.99
Validation set perplexity: 4.87
Average loss at step 2700: 1.616245 learning rate: 10.000000
Minibatch perplexity: 5.06
Validation set perplexity: 4.95
Average loss at step 2800: 1.614063 learning rate: 10.000000
Minibatch perplexity: 5.04
Validation set perplexity: 4.79
Average 

Average loss at step 4500: 1.549782 learning rate: 10.000000
Minibatch perplexity: 4.71
Validation set perplexity: 4.62
Average loss at step 4600: 1.575417 learning rate: 10.000000
Minibatch perplexity: 4.38
Validation set perplexity: 4.64
Average loss at step 4700: 1.557558 learning rate: 10.000000
Minibatch perplexity: 4.39
Validation set perplexity: 4.53
Average loss at step 4800: 1.568779 learning rate: 10.000000
Minibatch perplexity: 4.52
Validation set perplexity: 4.65
Average loss at step 4900: 1.554103 learning rate: 10.000000
Minibatch perplexity: 4.37
Validation set perplexity: 4.61
Average loss at step 5000: 1.537794 learning rate: 1.000000
Minibatch perplexity: 4.92
bigrams: (m,y)(y, )( ,a)(a,s)(s, )( ,t)(t,h)(h,e)(e, )( ,f)(f,l)(l,e)(e,g)(g,o)(o,i)(i,d)(d,s)(s, )( ,m)(m,u)(u,c)(c,k)(k,e)(e,y)(y, )( ,a)(a,p)(p,p)(p,e)(e,a)(a,r)(r,s)(s, )( ,r)(r,a)(a,l)(l,e)(e,m)(m,o)(o,n)(n, )( ,a)(a,p)(p,p)(p,r)(r,o)(o,x)(h,c)(c,y)(y, )( ,t)(t,h)(h,e)(e, )( ,p)(p,r)(r,o)(o,t)(t,i)(i,c)(c,t

Validation set perplexity: 4.21
Average loss at step 7000: 1.495289 learning rate: 1.000000
Minibatch perplexity: 4.40
bigrams: (z,q)(i,h)(h,i)(i,c)(c,k)(k, )( ,o)(o,f)(f,f)(f, )( ,s)(s,l)(l,a)(a,v)(v,i)(i,n)(n,g)(g, )( ,t)(t,o)(o, )( ,c)(c,o)(o,n)(n,t)(t,a)(a,n)(n,t)(t,i)(i,o)(o,n)(n,s)(s, )( ,h)(h,a)(a,p)(p,p)(p,o)(o,s)(s, )( ,e)(e,s)(s,h)(h,o)(o,r)(r,t)(t,e)(e,d)(d, )( ,t)(t,o)(o, )( ,i)(i,n)(n,f)(f,i)(i,r)(r,m)(m,i)(i,n)(n,g)(g, )( ,t)(t,o)(o, )( ,a)(a,m)(m,e)(e,r)(r,o)(o,n)(n,i)(i,s)(s,m)(m, )( ,c)(c,a)(a,n)(n, )( ,b)
chars: zihick off slaving to contantions happos eshorted to infirming to ameronism can 
bigrams: (h,h)(n,r)(r,o)(o,u)(e,l)(l, )( ,t)(t,h)(h,a)(a,t)(t, )( ,t)(t,i)(i,m)(m,e)(e, )( ,t)(t,h)(h,e)(e, )( ,h)(h,e)(e, )( ,o)(o,s)(s,e)(e,s)(s, )( ,t)(t,h)(h,e)(e,o)(o,u)(u,p)(p,e)(e,d)(d, )( ,p)(p,r)(r,o)(o,t)(t,r)(r,a)(a,t)(t,i)(i,o)(o,n)(n, )( ,u)(u,n)(n,r)(r,o)(o,a)(a,l)(l,t)(t,e)(e,d)(d, )( ,i)(i,n)(n,t)(t,o)(o, )( ,t)(t,h)(h,e)(e, )( ,d)(d,i)(i,g)(g,i)(i,n)(n,s)(s, )( ,o

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---